In [9]:
import os
import re
import numpy as np
import pandas as pd
from datetime import date, datetime
from calendar import monthrange
from IPython.display import display,HTML
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
#from DataProcessing import import_data,property_input,format_reservation
%run DataProcessing.ipynb
data = import_data()
property=property_input()

In [2]:
osbr = data[data["Listing"].str.contains("Cottage")]
mask = (osbr["checkin_date"]>=pd.to_datetime('2025-11-01')) & (osbr["checkin_date"]<=pd.to_datetime('2025-11-30')) |\
(osbr["checkout_date"]>=pd.to_datetime('2025-11-01')) & (osbr["checkout_date"]<=pd.to_datetime('2025-11-30'))
osbr = osbr.loc[mask]
checkins = osbr.assign(Date = osbr["checkin_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkin=("checkin_date",'count'))
checkouts = osbr.assign(Date = osbr["checkout_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkout=("checkout_date",'count'))
counts = checkins.merge(checkouts,on="Date",how="outer")
df = pd.DataFrame({
    "Date": pd.date_range("2025-11-01", "2025-11-30")
})
counts["Date"] = pd.to_datetime(counts["Date"])
counts = df.merge(counts,on="Date",how="left")
counts["Weekday"] = counts["Date"].dt.day_name()
counts["Date"] = counts["Date"].astype(str).str[:10]
counts = counts[["Date", "Weekday", "checkin", "checkout"]]
counts.to_excel("/Users/ylin/My Drive/Cohost/Data and Reporting/05-Cleaning/Brittany_booking_counts.xlsx", index=False,na_rep="")

In [14]:
idx=(osbr["checkout_date"]>=pd.to_datetime('2025-11-07')) & (osbr["checkout_date"]<=pd.to_datetime('2025-11-20'))
osbr.loc[idx,["Listing","checkout_date"]].merge(property[["Listing","BEDROOMS"]],on="Listing",how="left").groupby(["checkout_date","BEDROOMS"],as_index=False).count()

,checkout_date,BEDROOMS,Listing
0,2025-11-07,1.0,3
1,2025-11-07,2.0,1
2,2025-11-08,1.0,2
3,2025-11-08,2.0,3
4,2025-11-09,1.0,3
5,2025-11-09,2.0,4
6,2025-11-10,1.0,3
7,2025-11-11,1.0,1
8,2025-11-15,1.0,1
9,2025-11-16,2.0,1
